# Interactuando con Telegram


## Introducción

En este cuaderno vamos a crear un [*bot* de Telegram](https://core.telegram.org/bots) (un sencillo [programa](https://es.wikipedia.org/wiki/Bot) que interactúa con otros usuarios y se activa a través de mensajes) que, en la medida de lo posible, muestre ciertas capacidades lingüísticas. 

En definitiva, vamos a combinar una API para gestionar Telegram con las APIs de procesamiento lingüístico que hemos visto durante el curso.


## Creando un [*bot* de Telegram](https://core.telegram.org/bots)

Como en el caso del bot de Twitter, para desarrollar cualquier aplicación automática en Telegram necesitamos solicitar unas credenciales. Sigue los pasos:

1. Lo primero que tienes que hacer, si no lo has hecho ya, es instalar el cliente de [Telegram](https://telegram.org/) en tu móvil (tendrás que asociar tu cuenta a un número de teléfono). 

2. Una vez tengas una cuenta funcionando, entra en [la página de BotFather](https://telegram.me/botfather) y mándale un mensaje (*Send message*). Si te da problemas desde este ordenador (que probablemente es lo que pasará si no tienes un cliente de Telegram instalado), desde el navegador de tu móvil entra en la URL: https://telegram.me/botfather. Esta acción abrirá un hilo de conversación con @botfather. 

3. Solicita la creación de un nuevo bot con el comando `/newbot`. BotFather te pedirá que elijas un nombre para el bot y para el usuario del bot. Sigue las instrucciones. BotFather te proporcionará las credenciales del bot, un mensaje similar a:

    *The token is a string along the lines of 110201543:AAHdqTcvCH1vGWJxfSeofSAs0K5PALDsaw that will be required to authorize the bot and send requests to the Bot API.*

4. Toma nota del token de autenticación, lo necesitarás más adelante. Y, en principio, ya tienes todo lo que necesitas para crear tu *bot*, pero si necesitas algo más, revisa el resto de [funcionalidades (https://core.telegram.org/bots#botfather) de BotFather. 

Por mi cuenta, he creado un sencillo bot para la clase. Puedes hablar con él en https://t.me/progplnbot.

In [ ]:
# bot: progpln
# username: progplnbot
# url: https://t.me/progplnbot
# token: 304523995:AAF09xZgnX7cXQZr7u-CayCY19FweyfGD-0

## Primera versión: un bot respondón

Nuestro primer intento va a consistir en un bot que repite, automáticamente, cualquier mensaje que le recibe. El esqueleto del programa, el código mínimo necesario para que funciones, es el siguiente:


In [ ]:
import telebot

# credenciales
TOKEN = 'TUS CREDENCIALES' 
bot = telebot.TeleBot(TOKEN)


# función para responder cualquier mensaje que recibe
@bot.message_handler(func=lambda m: True)
def echo_all(message):
    bot.reply_to(message, message.text)
    

# instrucción para poner a la escucha al bot
bot.polling()

## Añadiendo instrucciones especiales

Podemos añadir funciones que permitan manejar instrucciones especiales, que se introducen con el prefijo `/intruccion`. Fíjate en esta celda, en la que definimos un saludo como respuesta a la instrucciones `/start` o `/saludo`.

In [ ]:
import telebot

# credenciales
TOKEN = 'TUS CREDENCIALES' 
bot = telebot.TeleBot(TOKEN)


# función para manejar la instrucción /start (o /saludo) 
@bot.message_handler(commands=['start', 'saludo'])
def send_welcome(message):
    bot.reply_to(message, '¡Hoooola! Soy un bot respondón :-) Dime algo')
    
    
# función para responder cualquier mensaje que recibe
@bot.message_handler(func=lambda m: True)
def echo_all(message):
    bot.reply_to(message, message.text)
    
    
# instrucción para poner a la escucha al bot
bot.polling()

## Funcionalidades extra a través de librerías

Añadimos otra función extra: en este caso, queremos que el bot nos diga la fecha y la hora actual como respuesta a la instrucciones `/ahora` o `/reloj`. Para ello usaremos también la [librería `datetime`](https://docs.python.org/3.0/library/time.html) que permite gestionar operaciones con horas y fechas.

In [ ]:
import telebot
from datetime import datetime

# credenciales
TOKEN = 'TUS CREDENCIALES' 
bot = telebot.TeleBot(TOKEN)


# función para manejar la instrucción /start (o /saludo) 
@bot.message_handler(commands=['start', 'saludo'])
def send_welcome(message):
    bot.reply_to(message, '¡Hoooola! Soy un bot respondón que te dice qué hora es :-) Prueba con /reloj')
    
    
# función para manejar la instrucción /ahora (o /reloj) 
@bot.message_handler(commands=['dimelahora'])
def show_time(message):
    ahora = datetime.now()
#    bot.reply_to(message, ahora.strftime('Son las %H:%M:%S del %d/%m/%Y.'))
    bot.reply_to(message, ahora.strftime('Ahora mismo tenemos las %H horas y %M minutos, caballero'))
    
    
# función para responder cualquier mensaje que recibe
@bot.message_handler(func=lambda m: True)
def echo_all(message):
    bot.reply_to(message, message.text)
    
    
# instrucción para poner a la escucha al bot
bot.polling()

## Incorporando funcionalidades lingüísticas

Por último, vamos a incorporar algo de procesamiento lingüístico y expresiones regulares a nuestro bot. En la celda siguiente, añadimos una nueva función para analizar la polaridad de mensajes. Para ello, capturamos mensajes que contengan la instrucción *polaridad* y una frase en inglés, usando expresiones regulares y el análisis de sentimiento de `textblob`.

Fíjate en que he cambiado el código utilizado para imprimir mensajes para hacerlos más limpios y evitar qu se imprima siempre el mensaje de entrada del usuario. He sustituido la instrucción `reply_to` de las celdas anteriores por `send_message`. El comportamiento es similar, se utiliza para que el bot imprima un mensaje pero: 1) como primer argumento es necesario especificar la propiedad `chat.id` del mensaje de entrada (en este ejemplo, la variable `m`); y 2) verás que la respuesta del bot es ahora más limpia.

In [ ]:
import telebot
from datetime import datetime
import re
from textblob import TextBlob

# credenciales
TOKEN = 'TUS CREDENCIALES' 
bot = telebot.TeleBot(TOKEN)


# función para manejar la instrucción /start 
@bot.message_handler(commands=['start'])
def send_welcome(m):
    user = m.from_user.first_name # otra novedad, de esta forma recupero el nombre del usuario
    welcometext = '''¡Hola %s! Soy un bot respondón :-) Sé hacer algunas cosas, prueba por ejemplo a:

- preguntarme que hora es con los comandos /ahora o /reloj
- pedirme que analice la polaridad de una frase en ingles con "polaridad Some sentence in English"
''' % (user)
    bot.send_message(m.chat.id, welcometext)
    
    
# función para manejar la instrucción /ahora (o /reloj) 
@bot.message_handler(commands=['ahora', 'reloj'])
def clock(m):
    ahora = datetime.now()
    bot.send_message(m.chat.id, ahora.strftime('Son las %H:%M:%S del %d/%m/%Y.'))

        
# regex para capturar mensajes cuya polaridad se quiere analizar
polaridadregex = re.compile(r'''^[Pp]olaridad (.+)''')

@bot.message_handler(regexp='^[Pp]olaridad .+')
def polaridad(m):
    opinion = polaridadregex.match(m.text).groups()[0]
    t = TextBlob(opinion)
    if t.sentiment.polarity <= -0.2:
        bot.send_message(m.chat.id, 'Eres gente mala :-( ')
    if -0.2 < t.sentiment.polarity <= 0.2:
        bot.send_message(m.chat.id, 'Tu mensaje es neutro :-| ')
    if 0.2 < t.sentiment.polarity:
        bot.send_message(m.chat.id, 'Tu mensaje es positivo :-) ')
        
        
# función para responder cuando no entiende el mensaje
@bot.message_handler(func=lambda f: True)
def echo_all(m):
    bot.send_message(m.chat.id, 'No entiendo lo que dices. Prueba a ejecutar /start.')

                
bot.polling()